In [ ]:
# Import the test server module
import test_jupyter_proxy
import os
from IPython.display import display, Markdown, HTML

In [ ]:
# Check if we're in a Jupyter/JupyterHub environment
print("Environment Check:")
print("=" * 50)

env_vars = [
    'JUPYTERHUB_SERVICE_PREFIX',
    'JUPYTERHUB_BASE_URL',
    'JUPYTERHUB_USER',
    'JUPYTERHUB_API_URL'
]

in_jupyterhub = False
for var in env_vars:
    value = os.environ.get(var, 'Not set')
    print(f"{var:30s}: {value}")
    if value != 'Not set':
        in_jupyterhub = True

print("=" * 50)
if in_jupyterhub:
    print("✅ Running in JupyterHub/Binder environment")
else:
    print("⚠️  Not in JupyterHub environment (local Jupyter)")
    print("   Proxy will still work, but URLs will be simpler")

In [ ]:
# Start the test server on port 9999
PORT = 9999
thread = test_jupyter_proxy.start_test_server(port=PORT, background=False)

print(f"\n✅ Test server started on port {PORT}")
print(f"   Server thread: {thread}")

In [ ]:
# Generate and display clickable links to access the test server
prefix = os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '')
if prefix and not prefix.endswith('/'):
    prefix += '/'

proxy_url = f"{prefix}proxy/{PORT}/"

html_output = f"""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
    <h2 style="margin-top: 0;">🚀 Test Server Ready!</h2>
    <p style="font-size: 16px;">Click the button below to open the test page:</p>
    <a href="{proxy_url}" target="_blank" 
       style="display: inline-block; background: #4CAF50; color: white; 
              padding: 15px 30px; text-decoration: none; border-radius: 5px; 
              font-weight: bold; margin: 10px 0;">
        📱 Open Test Server
    </a>
    <div style="background: rgba(255,255,255,0.2); padding: 10px; 
                border-radius: 5px; margin-top: 15px;">
        <strong>Proxy URL:</strong> <code>{proxy_url}</code><br>
        <strong>Direct URL:</strong> <code>http://localhost:{PORT}/</code>
    </div>
</div>
"""

display(HTML(html_output))

# Also show as markdown for copy-paste
display(Markdown(f"""
### Access URLs:

- **Via Proxy**: [{proxy_url}]({proxy_url})
- **Direct** (if local): [http://localhost:{PORT}/](http://localhost:{PORT}/)
"""))

## What to Test

After opening the test server page, you should see:

1. ✅ **Success message** - Confirms the proxy is working
2. **Connection details** - Shows the request path, port, and proxy prefix
3. **Interactive tests** - Buttons to test API endpoints and static resources
4. **Environment variables** - JupyterHub-specific environment details

### Troubleshooting

If the link doesn't work:
- Verify `jupyter-server-proxy` is installed: `pip show jupyter-server-proxy`
- Check that the server is running (see output above)
- Try accessing directly: `http://localhost:9999/`
- Check Jupyter server logs for proxy-related errors

In [ ]:
# Test the API endpoint programmatically
import requests

try:
    # Try direct localhost access first
    response = requests.get(f"http://localhost:{PORT}/api/test", timeout=5)
    print("✅ Direct API access successful!")
    print(f"   Status: {response.status_code}")
    print(f"   Response: {response.text}")
except Exception as e:
    print(f"❌ Direct API access failed: {e}")
    print("   This might be normal in some environments")

## Comparison with LDaCA Implementation

This test server uses the same proxy pattern as the LDaCA app:

### Backend (`ldaca_web_app_backend/deploy.py`)
```python
base = os.environ["JUPYTERHUB_SERVICE_PREFIX"]
url = f"{base}proxy/{port}/"
display(Javascript(f"window.open('{url}', '_blank');"))
```

### Frontend (`frontend/src/api/env.ts`)
```typescript
const match = pathname.match(PROXY_REGEX);
if (match) {
    const prefix = match[1];
    return `${origin}${prefix}${backendPort}/api`;
}
```

Both rely on:
1. `jupyter-server-proxy` to expose the `/proxy/<port>/` endpoint
2. `JUPYTERHUB_SERVICE_PREFIX` environment variable for correct URL construction
3. Pattern matching to detect and rewrite proxied paths

In [ ]:
# Show how to configure this server permanently in Jupyter
print("\n" + "="*60)
print("Optional: Permanent Server Configuration")
print("="*60)
test_jupyter_proxy.create_jupyter_server_config()

## Cleanup

The server runs in a daemon thread and will automatically stop when the kernel is shut down.

To stop it manually, restart the kernel: `Kernel → Restart`